In [ ]:
!pip install lightly

In [3]:
import torch
import torchvision
import pytorch_lightning as pl
import lightly
from torchmetrics.classification import Accuracy

In [5]:
import torch
torch.__version__

'1.11.0'

In [6]:
from torchvision.models import resnet101
from torch import nn
from torch.utils.data import DataLoader

In [7]:
import torchvision
from torchvision import datasets 
from torchvision.transforms import ToTensor

In [8]:
path_to_train ='Augmented Knee AP Dataset/Train'
path_to_test ='Augmented Knee AP Dataset/Val'

In [9]:
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((400,400))
])
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((400, 400))
])

In [10]:
train_set = datasets.ImageFolder(root=path_to_train, transform= train_classifier_transforms)
test_set = datasets.ImageFolder(root= path_to_test , transform = test_transforms)

In [11]:
train_dl = DataLoader(train_set, batch_size=16, shuffle=True)
test_dl = DataLoader(test_set, batch_size=16)

In [12]:
import pytorch_lightning as pl

In [13]:
class ResNetModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet101(num_classes=6)
    self.loss = nn.CrossEntropyLoss()
    self.accuracy = Accuracy()

  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    
    preds = torch.argmax(logits, dim = 1)
    acc = self.accuracy(preds, y)
    self.log('train_acc', acc,
             on_epoch=True, prog_bar=True)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = self.accuracy(preds, y)
    self.log('val_loss', loss, prog_bar=True)
    self.log('val_acc', acc, prog_bar=True)
    return loss

  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.05)

In [14]:
model = ResNetModel()

In [15]:
trainer = pl.Trainer(
    gpus=1,
    max_epochs=50
)
trainer.fit(model, train_dl, test_dl)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [16]:
trainer.save_checkpoint("ResNet101_0.pt")

In [17]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [18]:
from tqdm.autonotebook import tqdm

In [19]:
inference_model = ResNetModel.load_from_checkpoint("./ResNet101_0.pt")

In [20]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  preds, probs = get_prediction(x, inference_model)
  pred_y.extend(preds.cpu())

  0%|          | 0/11 [00:00<?, ?it/s]

In [21]:
from sklearn.metrics import classification_report

In [22]:
print(classification_report(true_y, pred_y, digits=2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      0.91      0.95        34
           2       0.88      1.00      0.93        21
           3       1.00      1.00      1.00        21
           4       1.00      0.90      0.95        20
           5       0.94      1.00      0.97        34

    accuracy                           0.97       163
   macro avg       0.97      0.97      0.97       163
weighted avg       0.97      0.97      0.97       163

